In [1]:
import pennylane as qml
import pennylane.numpy as np

In [2]:
from circuit import *

In [3]:
interface = 'autograd'

In [4]:
if interface == 'autograd':
    import pennylane.numpy as np
    dev = qml.device("default.qubit", wires=range(3))
    params = 0.05 * np.ones(18)

elif interface == 'jax':
    import jax
    import jax.numpy as jnp
    # next line raises TypeError with default.qubit.jax
    # dev = qml.device("default.qubit.jax", wires=range(3))
    dev = qml.device("default.qubit", wires=range(3))
    params = jnp.array([0.05] * 18)
elif interface == 'torch':
    import torch
    dev = qml.device("default.qubit", wires=range(3))
    params = torch.tensor([0.05] * 18, requires_grad=True)

In [5]:
@qml.qnode(dev, interface=interface)
def three_spins_forward(params, delta_time, n=1):
    three_spins_circuit(params)
    qml.ApproxTimeEvolution(hamiltonian(), delta_time, n)
    return qml.state()

@qml.qnode(dev, interface=interface)
def three_spins_current(params):
    three_spins_circuit(params)
    return qml.state()


In [6]:
# test this works

dt = 0.1
original_state = three_spins_current(params)
forward_state = three_spins_forward(params + 0.01, dt)

qml.math.fidelity(original_state, forward_state)

0.9750155801310574

In [8]:
prev_params = params
max_iterations = 50
recorded_params = [params]

def cost_function(params):
    
    dt = 0.1
    original_state = three_spins_current(prev_params)
    forward_state = three_spins_forward(params, dt)

    fidelity = qml.math.fidelity(original_state, forward_state)
    return 1 - fidelity


recorded_costs = [cost_function(params)]


In [9]:
cost_function(np.random.random(18))

0.817558134797245

In [12]:

opt = qml.GradientDescentOptimizer(stepsize=0.2)


for n in range(max_iterations):
    params, prev_cost = opt.step_and_cost(cost_function, params)
    
    recorded_costs.append(cost_function(params))
    recorded_params.append(params)
    
    if recorded_costs[-1] < 0.01:
        break
    

In [13]:
recorded_costs

[0.029201149605816323,
 0.01904860310164902,
 0.012354017689872765,
 0.00798208495930064]

In [14]:
recorded_params

[tensor([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
         0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05], requires_grad=True),
 tensor([0.06915227, 0.06870815, 0.06915341, 0.04961148, 0.04961135,
         0.04843418, 0.04663894, 0.04843171, 0.04770792, 0.04770773,
         0.06927037, 0.06932826, 0.06927175, 0.04762513, 0.04762483,
         0.05038708, 0.05050325, 0.05038476], requires_grad=True),
 tensor([0.08470357, 0.08390771, 0.08470566, 0.04916232, 0.04916205,
         0.04707891, 0.04377157, 0.04707443, 0.04579069, 0.04579035,
         0.08492368, 0.08505018, 0.08492619, 0.0455935 , 0.04559291,
         0.05055997, 0.05064761, 0.05055576], requires_grad=True),
 tensor([0.09726316, 0.09619314, 0.09726603, 0.04870496, 0.04870455,
         0.04592033, 0.04135112, 0.04591423, 0.04420073, 0.04420025,
         0.09757029, 0.09777132, 0.09757374, 0.04387037, 0.04386951,
         0.05059725, 0.05057625, 0.05059151], requires_grad=True)]